# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
from prettytable import PrettyTable

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory and getting all file paths 
#if you have open any CSV file in workspace, it will create a checkpoint version in folder - delete that file using below
#command in terminal rm -r event_data/.ipynb_checkpoints/

print(os.getcwd())

filepath = os.getcwd() + '/event_data'

for root, dirs, files in os.walk(filepath):
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)
    print(len(file_path_list))

/home/workspace
30


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file, read all csv files and append all data to one new 
# csv file event_datafile_new.csv
full_data_rows_list = [] 
    
for f in file_path_list:
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile:
        csvreader = csv.reader(csvfile) 
        next(csvreader)    
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            

#print(len(full_data_rows_list))
#print(full_data_rows_list)

csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
#Making a connection to a Cassandra instance your local machine (127.0.0.1)

from cassandra.cluster import Cluster
try:
    cluster = Cluster(['127.0.0.1'])
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [6]:
#Create a Keyspace 
try:
    session.execute("""CREATE KEYSPACE IF NOT EXISTS udacity WITH REPLICATION = {'class': 'SimpleStrategy',
    'replication_factor' : 1}""")
except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
#Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)
    

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [8]:
#inser_data function for all tables
def insert_data(query, session, values, dtypes, file):
    """
    Pipeline to insert data to a Apache Casssandra table from CSV file    
    Args:
        query (str): INSERT statement query
        session (object): Apache Casssandra session
        values (list): list of indicies to pass to INSERT statement
        dtypes (list): list of types to convert `values`
        file (str): CSV file path   
    Returns:
        None
    """
    assert(len(values) ==  len(dtypes))
    with open(file, encoding = 'utf8') as f:
        csvreader = csv.reader(f)
        next(csvreader) # skip header
        for line in csvreader:
            session.execute(query, tuple([dtype(line[x]) for x, dtype in zip(values, dtypes)]))

In [9]:
##creating a table song_details_in_session for query1
query = "CREATE TABLE IF NOT EXISTS song_details_in_session"
query = query + """(sessionid int, iteminsession int, artist text, song text, length decimal, PRIMARY KEY(sessionid, 
                     iteminsession) )"""
try:
    session.execute(query)
except Exception as e:
    print(e)
                    

In [10]:
#file name
file = 'event_datafile_new.csv'

#query
query = "INSERT INTO song_details_in_session (sessionid, iteminsession, artist, song, length)"
query = query + "VALUES (%s, %s, %s, %s, %s)"

#values and dataytpes
values = [8,3,0,9,5]
dtypes = [int, int, str, str, float]

#insert table function
insert_data(query, session, values, dtypes, file)

#### Do a SELECT to verify that the data have been inserted into each table

In [11]:
##Add in the SELECT statement to verify the data was entered into the table 
query = "SELECT artist, song, length FROM song_details_in_session WHERE sessionid= 338 AND iteminsession = 4"
try:
    rows = session.execute(query)
    t = PrettyTable(['Artist', 'Song', 'Length'])
    #df = pd.DataFrame(columns = ['artist', 'song', 'length'])#commented code to show results in DataFrame
    for row in rows:
        #print(row.artist, row.song, row.length)
        t.add_row([row.artist,row.song, row.length])
        #df1 = pd.DataFrame([{'artist' : row.artist, 'song' : row.song, 'length': row.length}])
        #print(df1)
        #df = df.append(df1, sort=True)
    #print(df)
    print(t)
except Exception as e:
    print(e)  


+-----------+---------------------------------+----------+
|   Artist  |               Song              |  Length  |
+-----------+---------------------------------+----------+
| Faithless | Music Matters (Mark Knight Dub) | 495.3073 |
+-----------+---------------------------------+----------+


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [12]:
##creating a table artist_song_details_per_user for query2
query = "CREATE TABLE IF NOT EXISTS artist_song_details_per_user"
query = query + """(userid int, sessionid int, iteminsession int, artist text, song text, firstname text, lastname text,
                     PRIMARY KEY(userid,sessionid,iteminsession) )"""
try:
    session.execute(query)
except Exception as e:
    print(e)                    

In [13]:
#file name
file = 'event_datafile_new.csv'

#query
query = "INSERT INTO artist_song_details_per_user (userid, sessionid, iteminsession, artist, song, firstname, lastname)"
query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"

#values and dtypes
values = [10,8,3,0,9,1,4]
dtypes = [int, int, int, str, str, str, str]

#insert-query
insert_data(query, session, values, dtypes, file)

In [14]:
##Add in the SELECT statement to verify the data was entered into the table
query = "SELECT artist, song, firstname, lastname FROM artist_song_details_per_user WHERE userid= 10 AND sessionid = 182"
try:
    rows = session.execute(query)
    t = PrettyTable(['artist', 'song', 'user'])
    #df = pd.DataFrame(columns = ['artist', 'song', 'firstname', 'lastname']) #commented code to show results in DataFrame
    for i, row in enumerate(rows):
        #print(row.artist, row.song, row.firstname, row.lastname)
        t.add_row([row.artist, row.song, row.firstname + ' ' +row.lastname])
        #df1 = pd.DataFrame([{'artist' : row.artist, 'song' : row.song, 'firstname': row.firstname, 'lastname':row.lastname}], index = [i])
        #df = df.append(df1, sort=True)
    #df['user'] = df['firstname'] + ' ' + df['lastname']
    #print(df[['artist','song','user']])
    print(t)
except Exception as e:
    print(e)

+-------------------+------------------------------------------------------+-------------+
|       artist      |                         song                         |     user    |
+-------------------+------------------------------------------------------+-------------+
|  Down To The Bone |                  Keep On Keepin' On                  | Sylvie Cruz |
|    Three Drives   |                     Greece 2000                      | Sylvie Cruz |
| Sebastien Tellier |                      Kilometer                       | Sylvie Cruz |
|   Lonnie Gordon   | Catch You Baby (Steve Pitron & Max Sanna Radio Edit) | Sylvie Cruz |
+-------------------+------------------------------------------------------+-------------+


In [15]:
##creating a table user_details_per_song for query3
query = "CREATE TABLE IF NOT EXISTS user_details_per_song"
query = query + """(song text, userid int, sessionid int, firstname text, lastname text, PRIMARY KEY(song, userid, sessionid) )"""

try:
    session.execute(query)
except Exception as e:
    print(e)               

In [16]:
#file_name
file = 'event_datafile_new.csv'
#query
query = "INSERT INTO user_details_per_song (song, userid, sessionid, firstname, lastname)"
query = query + "VALUES (%s, %s, %s, %s, %s)"

#values and dtypes
values = [9,10,8,1,4]
dtypes = [str, int, int, str, str]

#insert data
insert_data(query, session, values, dtypes, file)

In [17]:
##Add in the SELECT statement to verify the data was entered into the table
query = "SELECT firstname, lastname FROM user_details_per_song WHERE song = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
    t = PrettyTable(['User'])
    #df = pd.DataFrame(columns = ['firstname', 'lastname']) #commented code to show results in DataFrame
    for i, row in enumerate(rows):
        t.add_row([row.firstname + ' ' + row.lastname])
        #print(row.firstname, row.lastname)
        #df1 = pd.DataFrame([{'firstname': row.firstname, 'lastname':row.lastname}], index = [i])
        #df = df.append(df1, sort=True)
    #df['user'] = df['firstname'] + ' ' + df['lastname']
    #print(df[['user']])
    print(t)
except Exception as e:
    print(e)

+------------------+
|       User       |
+------------------+
| Jacqueline Lynch |
|   Tegan Levine   |
|   Sara Johnson   |
+------------------+


### Drop the tables before closing out the sessions

In [18]:
##Drop all the tables before closing out the sessions
queries = ['song_details_in_session', 'artist_song_details_per_user', 'user_details_per_song']

for i in queries:
    query = "drop table " +  i
    try:
        rows = session.execute(query)
    except Exception as e:
        print(e)

### Close the session and cluster connection¶

In [19]:
session.shutdown()
cluster.shutdown()